# Agent to play Tic-Tac-Toe

## Time to create an agent with some sense.

Rather than choosing random moves, the agent will decide the best move going forward. It will choose the best move from all the possible moves. This is generally how newcomers play any game, trying to optimise the immediate action.

![TicTacToe](tictactoe.png)

### Import packages

In [2]:
import random
import numpy as np
from IPython.display import clear_output
from time import sleep
from util import *

### Let's create the battlefield!

In [3]:
ttt = np.zeros((3,3), np.uint16)
ttt

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=uint16)

In [4]:
print(ttt.shape)
print(ttt.reshape(3,3))
print(type(ttt))

(3, 3)
[[0 0 0]
 [0 0 0]
 [0 0 0]]
<class 'numpy.ndarray'>


### Create the necessary methods for playing

In [5]:
def get_valid_moves(ttt):
    """
    Get positions in the grid that are empty.
    
    Parameters:
    ===========
    ttt: numpy.ndarray
        --> Tic-Tac-Toe grid.
    
    Returns:
    ========
    valid_moves: list
        --> Grid positions which are empty.
    """
    grid = ttt.copy()
    if grid.shape != (9,):
        grid = grid.reshape(9)
    valid_moves = [x for x in range(9) if grid[x] == 0]
    return valid_moves

In [6]:
assert get_valid_moves(np.array([0,0,0,1,0,1,1,1,0], np.uint16)) == [0,1,2,4,8]
assert get_valid_moves(np.array([[0,1,0],[1,1,1],[0,1,1]], np.uint16)) == [0,2,6]

In [7]:
print(dir())

['In', 'Out', '_', '_3', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i2', '_i3', '_i4', '_i5', '_i6', '_i7', '_ih', '_ii', '_iii', '_oh', 'agent1', 'agent2', 'check_win', 'clear_output', 'exit', 'get_ipython', 'get_random_tile', 'get_valid_moves', 'ngames', 'np', 'print_result', 'quit', 'random', 'result', 'sleep', 'ttt']


In [8]:
def check_window(window, seq, player):
    """Count the player tiles in the concerned window and the remaining tiles should be empty.
    This will ensure that the player can play further in those tiles.
    
    For example,
    
         1 2 3  |  1 2 3
      1  X - X  |  - - X
      2  O - -  |  O - X
      3  O X O  |  O X -
    
    Let's consider the first column(X O O), if we were to check for this
    window, we can see that 'O' has better outcome with 2 of its pieces in
    a sequence, however, since there are no further tiles to play and win
    in this window, this should not be awarded high score.
    
    Moreover, in the second grid, the first column is much better to play
    a piece by the player "O", since there is an empty tile where further
    play is possible.
    
    Parameters:
    ===========
    window: list
        --> consecutive tiles to score
        
    seq: int
        --> number of same pieces to count
        
    player: int
        --> the player piece to count, eg: 1(X) or 2(O)
    
    Return:
    =======
    :boolean
    
    """
    return window.count(player) == seq and window.count(0) == 3-seq

def count_windows(grid, seq, player):
    """
    Counts the number of windows that satisfy the given parameters,
    i.e. for the given player and the amount of pieces in a window.
    
    Parameters:
    ===========
    grid: numpy.ndarray
        --> 3x3 Tic Tac Toe grid
    
    seq: int
        --> number of same pieces to count
    
    player: int
        --> the player piece to count, eg: 1(Player 1) or 2(Player 2)
    
    Return:
    =======
    windows: int
        --> number of windows that satisfy the given criteria.
    """
    windows = 0
    # Horizontal(row)
    for row in grid:
        if check_window(list(row), seq, player):
            windows += 1
    
    # Vertical(column)
    for col in range(3):
        if check_window(list(grid[:,col]), seq, player):
            windows += 1
    
    # Diagonal
    window = list(grid[range(3), range(3)])
    if check_window(window, seq, player):
        windows += 1
    window = list(grid[range(2,-1,-1), range(2,-1,-1)])
    if check_window(window, seq, player):
        windows += 1
    return windows

### One Step Lookahead

Now that we have the necessary code for one step lookahead algorithm, let me explain how the algo works

#### Algo(Simplified)
1. While game not done:
    1. For each valid move from valid moves:
        - play the move
        - evaluate the play and assign a score
    1. Choose the best solution and play that move

#### Explaination
The algo above did not help much with understanding the actual scoring and how it can be helpful. In this section we will look at how scoring works and how it can be helpful.

##### Example 1:
| |1|2|3|
|-|-|-|-|
|1|X|O|X|
|2|X|-|O|
|3|-|X|O|

Consider all the possible moves **O** can play: (3,1) or (2,2). The best move would be to play at (1,3).

##### Example 2:

| |1|2|3|
|-|-|-|-|
|1|X|-|O|
|2|X|O|-|
|3|-|O|-|

Consider all the possible moves **X** can play: (1,2), (2,3), (3,1) and (3,3). The best move in this case would be to play at (3,1) to win the game.

From the above two examples we have seen that it is necessary to assign each move with the correct score so that only the optimum choice is made. In the above example, we don't want to play at (1,2) to stop **O** from winning when we could easily win in the same move.


## Scoring each move:

- +1000 if won i.e XXX or OOO
- +1 if next move will win the game i.e, XX or OO
- -100 if opponent will win in next move i.e it already has a sequence of 2.

In [9]:
def get_heuristic(grid, player):
    """
    Evaluates move and assigns a score.
    
    Parameters:
    ===========
    grid: numpy.ndarray
        --> 3x3 Tic-Tac-Toe grid
    
    player: int
        --> 1(X) or 2(O) to denote the player
    
    Returns:
    ========
    score: int
        --> score the game board received
    """
    num_threes = count_windows(grid, 3, player)
    num_twos = count_windows(grid, 2, player)
    num_twos_opp = count_windows(grid, 2, player%2 + 1)
    score = 1e3*num_threes + num_twos - 1e2*num_twos_opp
    return score

In [10]:
ttt = np.array([
    [1,0,1],
    [0,2,0],
    [0,2,1]
], np.uint16)
ttt

array([[1, 0, 1],
       [0, 2, 0],
       [0, 2, 1]], dtype=uint16)

In [11]:
def score_move(grid, move, player):
    ttt = grid.copy()
    if ttt.shape != (9,):
        ttt = ttt.reshape(9)
    if ttt[move] != 0:
        raise "Invalid Move"
    ttt[move] = player
    return get_heuristic(ttt.reshape((3,3)), player)

In [25]:
def get_optimal_move(grid, player):
    valid_moves = get_valid_moves(grid)
    scores = []
    for valid_move in valid_moves:
        scores.append(score_move(grid, valid_move, player))
    max_score = max(scores)
    max_score_indices = [index for index,score in enumerate(scores) if score == max_score]
    return valid_moves[random.choice(max_score_indices)]

In [26]:
agent1 = {'piece':1}
agent2 = {'piece':2}

result = {'agent1':0, 'agent2':0, 'draws':0, 'total_games':0}
ngames = 100

print("Starting Game!")
for _ in range(ngames):
    # Reset the grid
    ttt = np.zeros((3,3), np.uint16)
#     print(ttt)
    result['total_games'] += 1
    for iteration in range(9):
        # Optimal Agent
        if iteration%2 == 0:
#             print("Optimal player")
            move = get_optimal_move(ttt, agent1['piece'])
#             print(f'Move: {move}')
            ttt.reshape(9)[move] = agent1['piece']
        # Dumb Agent
        else:
#             print("Random Player")
            move = get_random_tile(ttt)
            ttt[move] = agent2['piece']
        winner = check_win(ttt)
#         print(ttt)
#         sleep(1)
        clear_output()
        if winner is not None:
            if winner == 1:
                result['agent1'] += 1
            else:
                result['agent2'] += 1
            break
    if winner is None:
        result['draws'] += 1
print_result(result)

                      Stats                       
--------------------------------------------------
Total Games:100   Agent1:98    Agent2:1     Draws:1    


In [31]:
agent1 = {'piece':1}
agent2 = {'piece':2}

result = {'agent1':0, 'agent2':0, 'draws':0, 'total_games':0}
ngames = 100

print("Starting Game!")
for _ in range(ngames):
    # Reset the grid
    ttt = np.zeros((3,3), np.uint16)
    result['total_games'] += 1
    for iteration in range(9):
        # Optimal Agent 1
        if iteration%2 == 0:
            move = get_optimal_move(ttt, agent1['piece'])
            ttt.reshape(9)[move] = agent1['piece']
        # Optimal Agent 2
        else:
            move = get_optimal_move(ttt, agent2['piece'])
            ttt.reshape(9)[move] = agent2['piece']
        winner = check_win(ttt)
#         print(ttt)
#         sleep(1)
#         clear_output()
        if winner is not None:
            if winner == 1:
                result['agent1'] += 1
            else:
                result['agent2'] += 1
            break
    if winner is None:
        result['draws'] += 1
print_result(result)

Starting Game!
                      Stats                       
--------------------------------------------------
Total Games:100   Agent1:30    Agent2:24    Draws:46   
